In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [3]:
# for artificial nns in pytorch the inputs should be 2D

In [4]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 2,  3,  0,  7, 10,  1,  9,  5,  8]), tensor([6, 4]))

In [5]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [6]:
# torch module allows to fit a linear model with torch.nn

In [7]:
linear_model = nn.Linear(1,1) # this says we have one input feature and one target feature

In [8]:
linear_model(t_un_val)

tensor([[2.9953],
        [4.6906]], grad_fn=<AddmmBackward0>)

In [9]:
# we can check for the initial values of the parameters - now observe here that we do not have to explicitly specify parameters

In [11]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.7568]], requires_grad=True),
 Parameter containing:
 tensor([0.4297], requires_grad=True)]

In [15]:
# or we can do bias and weights separately

In [16]:
linear_model.bias

Parameter containing:
tensor([0.4297], requires_grad=True)

In [19]:
linear_model.weight

Parameter containing:
tensor([[0.7568]], requires_grad=True)

In [12]:
# verify the above values are indeed correct

In [13]:
t_un_val

tensor([[3.3900],
        [5.6300]])

In [20]:
0.7568 * 3.3900 + 0.4297, 0.7568 * 5.6300 + 0.4297 # we get the values above

(2.9952520000000002, 4.6904840000000005)

In [21]:
# we need not define the loss function either now - follows is a complete training loop

In [22]:
def training_loop(n_epochs, optimizer, model, loss, tu_train, tu_val, tc_train, tc_val):

    # define the for loop here
    for epoch in range(1, n_epochs + 1):
        tp_train = model(tu_train)
        loss_train = loss(tp_train, tc_train)

        tp_val = model(tu_val)
        loss_val = loss(tp_val, tc_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}," f" Validation loss {loss_val.item():.4f}")

In [23]:
linear_model = nn.Linear(1, 1) # <1>
# notice below how the parameters are now input to the optimizer
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

In [28]:
training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    # we do not have to separately define the mse/loss either
    loss = nn.MSELoss(),
    tu_train = t_un_train,
    tu_val = t_un_val, 
    tc_train = t_c_train,
    tc_val = t_c_val)

Epoch 1, Training loss 357.0454, Validation loss 136.8881
Epoch 1000, Training loss 3.6134, Validation loss 2.7016
Epoch 2000, Training loss 2.6120, Validation loss 4.4475
Epoch 3000, Training loss 2.5796, Validation loss 4.9092


In [30]:
print(linear_model.weight)

Parameter containing:
tensor([[5.5374]], requires_grad=True)


In [31]:
print(linear_model.bias)

Parameter containing:
tensor([-18.2208], requires_grad=True)
